In [ ]:
# !pip install chromadb
# !pip install langchain
# !pip install sentence_transformers
#!pip install sodapy
# !pip install langchain-experimental
#!pip install tiktoken
# !pip install pgvector

In [ ]:
import geopandas as gpd
import pandas as pd
import psycopg2
import geoalchemy2
from psycopg2 import sql
from sqlalchemy import create_engine
pd.set_option('display.max_columns', None)
import sodapy
from langchain.document_loaders import GeoDataFrameLoader
from langchain.vectorstores import Chroma
import langchain_experimental
from langchain.vectorstores.pgvector import PGVector
import openai
import chromadb
from chromadb.config import Settings
import numpy as np
import os
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores.pgvector import DistanceStrategy
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import utils as chromautils
from dotenv import load_dotenv
pd.set_option('display.max_columns', None)
load_dotenv()


In [ ]:
conn_params = {
    'user': os.environ.get('POSTGRES_USER'),
    'password': os.environ.get('POSTGRES_PASSWORD', ''),
    'host': os.environ.get('POSTGRES_HOST', ''),
    'port': os.environ.get('POSTGRES_PORT', '')
}

In [ ]:
dbname = 'threatanalysis'

In [ ]:
OPENAI_API_KEY =  os.environ.get('OPENAI_API_KEY')
OPENAI_API_KEY

In [ ]:
CONNECTION_STRING = f"postgresql+psycopg2://{conn_params['user']}:{conn_params['password']}@{conn_params['host']}:{conn_params['port']}/{dbname}"

In [ ]:
connection = psycopg2.connect(**conn_params)

In [ ]:
# Define the SQL query to fetch data from the table
table_name = "eventdata"
sql_query = f"""SELECT * FROM {table_name};"""

# Use Pandas to read the query result into a DataFrame
EventData = gpd.read_postgis(sql_query, connection, geom_col='geometry')

In [ ]:
table_name = "countydata"
sql_query = f"""SELECT * FROM {table_name};"""

# Use Pandas to read the query result into a DataFrame
countydata = gpd.read_postgis(sql_query, connection, geom_col='geometry')

In [ ]:
EventData.fillna('N/A', inplace=True)
EventData['event_date'] = EventData['event_date'].apply(lambda x: x.timestamp())
EventData['event_source'] = EventData['event_source'].apply(np.array)

In [ ]:
EventData.dtypes

In [ ]:
type(EventData['geometry'].iloc[0])

In [ ]:
loader = GeoDataFrameLoader(data_frame=EventData, page_content_column='geometry')

In [ ]:
docs = loader.load()

In [ ]:
llm = OpenAI(api_key=OPENAI_API_KEY, model_name="text-davinci-003")
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
docs = chromautils.filter_complex_metadata(docs)
db = Chroma.from_documents(docs, embeddings, persist_directory="./db")

In [ ]:
# Access the content and metadata of each document
for document in docs:
    content = document.page_content
    metadata = document.metadata

In [ ]:
# # Create a PGVector instance to house the documents and embeddings
# import pgvector
# from langchain.vectorstores.pgvector import DistanceStrategy
# from langchain.vectorstores.pgvector import PGVector
# from langchain.vectorstores import utils as chromautils

# docs = chromautils.filter_complex_metadata(docs)
# db = PGVector.from_documents(
#     documents= docs,
#     embedding = embeddings,
#     collection_name= "event_data",
#     distance_strategy = DistanceStrategy.COSINE,
#     connection_string=CONNECTION_STRING)


In [ ]:
# import chromadb
# # vector store settings
# VECTOR_STORE_PATH = r'../02_Data/00_Vector_Store'
# COLLECTION_NAME = 'my_collection'

# # Load embeddings_df.csv into data frame
# embeddings_df = EventData

# def get_or_create_client_and_collection(VECTOR_STORE_PATH, COLLECTION_NAME):
#     # get/create a chroma client
#     chroma_client = chromadb.PersistentClient(path=VECTOR_STORE_PATH)

#     # get or create collection
#     collection = chroma_client.get_or_create_collection(name=COLLECTION_NAME)

#     return collection

# # get or create collection
# collection = get_or_create_client_and_collection(VECTOR_STORE_PATH, COLLECTION_NAME)

In [ ]:
# db.get()

In [ ]:
# result = db.similarity_search("what is the event_id for event_city Coconino ", k=1000)
# result

In [ ]:
# client = chromadb.PersistentClient(path="db/")
#collection = client.create_collection(name="Threat")
# retriever = db.as_retriever()
#qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
# query = 'what is the event_date for event_id 01101'
# qa.run(query)

In [ ]:
# text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=10)
# texts = text_splitter.split_documents(docs)
# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

#embedding_function = HuggingFaceEmbeddings(
# model_name="sentence-transformers/all-MiniLM-L6-v2"
#) 
# chromadb_index = Chroma.from_documents(
#  texts, embedding_function, persist_directory='./input'
# )
# retriever = chromadb_index.as_retriever()